# LightGBM & 과소적합,과대적합 & validation set의 중요성

- Boosting은 여러개의 트리(혹은 다른 모델)를 만들되, 랜덤포레스트나 배깅과 같은 기법과는 다른게 기존에 있는 예측기를 조금씩 발전시켜서 이를 합한다는 개념이다.

- 랜덤포레스트는 병렬로 무지막지하게 많은 다양한 결정트리를 동시에 만든다면 부스팅은 점진적으로 Decision Tree를 발전시킨뒤에 이를 통합하는 과정을 거친다고 보면된다. 부스팅은 보통 두가지 방향이 있다.

- 정답지와 오답지간의 차이를 훈련에 다시 투입하여 gradient를 적극 이용해서 모델을 개선하는 방식. XGboost나 lightGBM이 여기에 속한다.

- GBM이란? : Gradient Boosting Machine(GBM)은 틀린부분에 가중치를 더하면서 진행하는 알고리즘

- LGBM은 머신러닝 부스팅 알고리즘의 성능을 높인것이라고 할 수 있다.

- 검증셋을 활용해서 검증셋에 대한 결과가 좋아지지 않을때까지 의사결정트리를 추가한다.

In [3]:
!pip install lightgbm

  Using cached lightgbm-3.1.0-py2.py3-none-win_amd64.whl (751 kB)


In [4]:
import lightgbm as lgb

In [5]:
import random
random.seed(0) # 어느 컴퓨터에서 추출해도 똑같은 값이 나오도록 고정
idx = random.sample(range(5121), 50) # 5000개의 컬럼중에 랜덤하게 50개를 추출

In [20]:
from sklearn.model_selection import StratifiedKFold 
# 클래스 개수가 불균등하게 분포가 되어있기 때문에 train이랑 validation을 나눌때 균등하게 나누기 위해 StratifiedKFold를 사용
skf = StratifiedKFold(n_splits=5) # n_splits=5 : 8대2로 나눔

for trn_idx, val_idx in skf.split(X_train, y_train):
    
    X_trn, X_val, y_trn, y_val = X_train.iloc[trn_idx], X_train.iloc[val_idx], y_train.iloc[trn_idx], y_train.iloc[val_idx]
    
    break # break -> for문을 다 돌리지 않고 1번만 돌림.

In [21]:
model_lgb = lgb.LGBMClassifier(n_estimators=50, learning_rate=0.03) 
# 분류문제이므로LGBMClassifier를 가져온다.
# learning_rate, n_estimators 등 이런것들이 사용자가 직접 설정해야하는 옵션인 '하이퍼파라미터' 

In [22]:
start = time.time()
model_lgb.fit(X_trn.iloc[:,idx], y_trn)# 여기서는 validaion셋을 넣지 않아서 n_estimators=50을 모두 학습한다. -> 안좋음. #fit, predict함수는 그대로 scikit-learn처럼 사용, #idx -> 위에서 랜덤하게 뽑은 50개의 칼럼만 가지고
time.time() - start

26.938925981521606

- early stopping
- 너무 적게 학습하면 즉 의사결정트리 수 n_estimators를 조금 주면 학습이 덜 됨 : 과소적합
- 너무 많이 학습하면 noise까지 학습해버리기 때문에 더 이상한 모델이 나옴. 새로 들어오는 test데이터에 잘 맞아야 하는데 train 데이터에만 너무 학습되어서 망함 : 과대적합

- **과소적합과 과대적합**

![](https://datascience.foundation/img/pdf_images/underfitting_and_overfitting_in_machine_learning_some_degrees_and_train_our_model.png)

- **검증 셋 활용**

![](https://datavedas.com/wp-content/uploads/2018/04/image003.jpg)




- **Training Error vs Validation Error**

  - 주황색 선이 validation error라고 봐주시길 바랍니다.

![](https://www.textbook.ds100.org/_images/bias_cv_train_test_error.png)

In [23]:
lgb_pred = model_lgb.predict_proba(X_val.iloc[:,idx])

In [24]:
model_lgb = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.03, objective='multiclass')

In [25]:
len(idx) 
#-> 학습 시간 줄이려고(너무 오래걸리므로 just for example)

50

In [26]:
start = time.time()
model_lgb.fit(X_trn.iloc[:,idx], y_trn, 
              eval_set=(X_val.iloc[:,idx], y_val), 
              verbose = 10, #verbose = 10 : 10번 반복할 때마다 logloss값을 보여준다.
              early_stopping_rounds=10) # early_stopping_rounds은 범위를 잡는 것, early_stopping_rounds를 조정해서 validation error가 최소가 되는 지점에 학습을 멈추도록
time.time() - start 
# validation 셋을 활용해서 원하는 지점까지 학습 함.
# n_estimators=100으로 설정해서 10, 20, 30.. 가다가 40~50사이에 loss값이 증가해서 42지점에서 알아서 멈춤. 
# 즉 validaion의 loss값이 최소가 될때 stop
# 이런식으로 하이퍼파라미터를 조정할 수도 있음.

Training until validation scores don't improve for 10 rounds
[10]	valid_0's multi_logloss: 4.26184
[20]	valid_0's multi_logloss: 4.14101
[30]	valid_0's multi_logloss: 4.08661
[40]	valid_0's multi_logloss: 4.0672
[50]	valid_0's multi_logloss: 4.07214
Early stopping, best iteration is:
[42]	valid_0's multi_logloss: 4.06639


32.25992274284363